In [1]:
# 导入数据
import pandas as pd
df1 = pd.read_excel(r'D:\研究生资料\玛氏数据\5-22\chengdu_521.xlsx')
df_POI = pd.read_excel(r'D:\研究生资料\玛氏数据\chengdu_poi_category.xlsx')

## 4.按销量占比的门店分析

### a. 占比划分：>=70%（金额；强），30%<=占比<70%（中）；占比<30%（弱）


（1）	求出每个门店中三个品类的销量，然后求各个品类销量的占比；

In [41]:
import numpy as np

In [44]:
# 求出每个门店的总销量
storeid=list(df1['store_code'])
storeid=list(set(storeid))

id_sum=[] 
for i in range(len(storeid)):
    demo1 = df1.loc[df1['store_code']==storeid[i]]
    aa = list(demo1['case_ordered'])
    sum_1=np.array(aa).sum()
    id_sum.append(sum_1)

In [45]:
print(len(storeid))
print(len(id_sum))

16784
16784


In [47]:
# 求出每个门店的Chocolate\Chewing\Sugar总销量
id_cho_sum=[] 
id_che_sum=[] 
id_sugar_sum=[]
for i in range(len(storeid)):
    demo1 = df1.loc[df1['store_code']==storeid[i]]
    aa = list(demo1.loc[demo1['ProductCategory']=='Chocolate','case_ordered'])
    bb = list(demo1.loc[demo1['ProductCategory']=='Chewing/Bubble Gum','case_ordered'])
    cc = list(demo1.loc[demo1['ProductCategory']=='Sugar','case_ordered'])
    sum_1=np.array(aa).sum()
    sum_2=np.array(bb).sum()
    sum_3=np.array(cc).sum()
    id_cho_sum.append(sum_1)
    id_che_sum.append(sum_2)
    id_sugar_sum.append(sum_3)


In [48]:
print(id_sum[0])
print(id_cho_sum[0]+id_che_sum[0]+id_sugar_sum[0])

2.124999999995
2.124999999995


In [49]:
cho_pro = [id_cho_sum[i]/id_sum[i] for i in range(len(id_sum))]
che_pro = [id_che_sum[i]/id_sum[i] for i in range(len(id_sum))]
sug_pro = [id_sugar_sum[i]/id_sum[i] for i in range(len(id_sum))]

In [61]:
# 取小数点后四位
cho_pro_1=['%.4f' % cho_pro[i] for i in range(len(cho_pro))]
cho_pro_2=[float(cho_pro_1[i]) for i in range(len(cho_pro_1))]
che_pro_1=['%.4f' % che_pro[i] for i in range(len(che_pro))]
che_pro_2=[float(che_pro_1[i]) for i in range(len(che_pro_1))]
sug_pro_1=['%.4f' % sug_pro[i] for i in range(len(sug_pro))]
sug_pro_2=[float(sug_pro_1[i]) for i in range(len(sug_pro_1))]

In [64]:
data1={
    '门店编号':storeid,
    'Chocolate销售占比':cho_pro_2,
    'Gum销售占比':che_pro_2,
    'Sugar销售占比':sug_pro_2 
}
data1=pd.DataFrame(data1)
data1.to_excel(r'C:\Users\Administrator\Desktop\Store_Volume_Tag_by_Cat_RATIO.xlsx')

In [65]:
data1.head(3)

,门店编号,Chocolate销售占比,Gum销售占比,Sugar销售占比
0,50954241,0.6078,0.3922,0.000
1,50954245,0.0439,0.0481,0.908
2,50987014,0.0000,1.0000,0.000


（2）	根据每个门店各个品类金额的占比，将其打上强、中、弱的标签；
Store_Volume_Tag_by_Cat 可以把3个类目排成一个表把每个门店每个品类的比例放在一起。

（3）	生成一张各个门店三个品类强中弱的统计表（b的结果）；


（4）	统计出每个品类的强、中、弱分别对应的门店编号

## c. 按case量累计出强门店的top15 (注意，这里的top15是在门店前10 POI的case累加得到的case排序前15)


（1）	求出巧克力强门店每个门店去重后排序TOP10的POI；


（2）	求出巧克力强门店每个门店巧克力的case量；   
（3）	将（2）中case量分别加到每个门店对应的POI上；   
（4）	将相同的POI的case量相加，然后排序；   
（5）	选择累积量最大的前15 POI;   
（6）	糖果和口香糖的计算过程如上

In [67]:
df_22 = pd.read_excel(r'C:\Users\Administrator\Desktop\Store_Volume_Tag_by_Cat(1).xlsx')

In [69]:
df_22.head(3)

,Unnamed: 0,门店编号,Chocolate销售占比,Chocolate-Tag,Gum销售占比,Gum-Tag,Sugar销售占比,Sugar-Tag
0,0,50954241,0.6078,中,0.3922,中,0.000,弱
1,1,50954245,0.0439,弱,0.0481,弱,0.908,强
2,2,50987014,0.0000,弱,1.0000,强,0.000,弱


In [85]:
df_POI_zong = pd.read_excel(r'C:\Users\Administrator\Desktop\任务一\All_Store_POI纵.xlsx')

In [92]:
# 取出Chocolate-Tag为强的门店编号
id_cho_1 = list(df_22.loc[df_22['Sugar-Tag']=='强','门店编号'])

In [93]:
demo1_POI=df_POI_zong.loc[df_POI_zong['storecode'].isin(id_cho_1)]

In [94]:
demo1_POI_id = list(demo1_POI['storecode'])
demo1_POI_id = list(set(demo1_POI_id))
# demo1_POI_id门店巧克力销售加总
demo1_POI_id_cho_sum=[] 
for i in range(len(demo1_POI_id)):
    uu = df1.loc[df1['store_code']==demo1_POI_id[i]]
    sum_1 = uu.loc[uu['ProductCategory']=='Sugar','case_ordered'].sum()
    demo1_POI_id_cho_sum.append(sum_1)

In [95]:
# demo1_POI赋上销量信息
# 构建门店编号及其销量信息的表
data3={
    'demo1_POI_id':demo1_POI_id,
    'demo1_POI_id_cho_sum':demo1_POI_id_cho_sum
}
data3=pd.DataFrame(data3)
data3.sort_values(by='demo1_POI_id',inplace=True)

k5_1=demo1_POI
k5_1.sort_values(by='storecode',inplace=True)

sum_2=list(data3['demo1_POI_id_cho_sum'])
s_id =list(data3['demo1_POI_id'])

sum_3 =[]
for i in range(len(s_id)):
    cc=len(k5_1.loc[k5_1['storecode']==s_id[i]])
    for j in range(cc):
        sum_3.append(sum_2[i])

k5_1['总销量']=sum_3

E:\My large programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
E:\My large programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [96]:
# k5_1相同POI销量相加
k5_4 = k5_1.groupby(['s_cat_id','s_cat_value'])['总销量'].sum()
k5_4 = k5_4.reset_index()
k5_4.sort_values(['总销量'],ascending=[0],inplace=True)
tt= list(k5_4['s_cat_id'])
r1=[]
r2=[]
r3=[]
r4=[]
for i in range(len(tt)):
    r1.append(list(set(list(k5_1.loc[k5_1['s_cat_id']==tt[i],'b_cat_id'])))[0])
    r2.append(list(set(list(k5_1.loc[k5_1['s_cat_id']==tt[i],'b_cat_value'])))[0])
    r3.append(list(set(list(k5_1.loc[k5_1['s_cat_id']==tt[i],'m_cat_id'])))[0])
    r4.append(list(set(list(k5_1.loc[k5_1['s_cat_id']==tt[i],'m_cat_value'])))[0])
k5_4['b_cat_id']=r1
k5_4['b_cat_value']=r2
k5_4['m_cat_id']=r3
k5_4['m_cat_value']=r4

k5_4=k5_4[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

In [97]:
# POI_Chocolate_Strong
k5_4.to_excel(r'C:\Users\Administrator\Desktop\POI_Sugar_Strong.xlsx')

In [98]:
#Top_15_POI_Chocolate_Strong
k5_4_top15=k5_4.head(15)
k5_4_top15.to_excel(r'C:\Users\Administrator\Desktop\Top_15_Sugar_Gum_Strong.xlsx')

## d. 按出现次数统计出弱品类门店的top10



（1）	求出巧克力弱门店每个门店去重后排序TOP10的POI；  
（2）	统计出（1）中每个POI出现的次数，降序排序；  
（3）	选取（2）中出现次数做多的10个POI；  
（4）	糖果和口香糖的计算过程如上

In [107]:
# 取出Sugar为弱的门店编号,并从df_POI_zong中取出对应的POI
id_sug_1 = list(df_22.loc[df_22['Gum-Tag']=='弱','门店编号'])
demo1_POI=df_POI_zong.loc[df_POI_zong['storecode'].isin(id_sug_1)]

In [108]:
# 统计POI出现次数，降序
k1_1 = demo1_POI
k1_1['nums']=0
uu_1=list(k1_1.groupby(['s_cat_id']).count()['nums'].index) # 表示s_cat_id
uu_2=list(k1_1.groupby(['s_cat_id']).count()['nums'].values) # 表示出现次数
uu_3=[]
uu_4=[]
uu_5=[]
uu_6=[]
uu_7=[]
# 'b_cat_id','b_cat_value','m_cat_id','m_cat_value'
for i in range(len(uu_1)):
    f1=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'b_cat_id'])[0]
    f2=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'b_cat_value'])[0]
    f3=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'m_cat_id'])[0]
    f4=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'m_cat_value'])[0]
    f5=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'s_cat_value'])[0]
    uu_3.append(f1)
    uu_4.append(f2)
    uu_5.append(f3)
    uu_6.append(f4)
    uu_7.append(f5)

data3={
    'b_cat_id':uu_3,
    'b_cat_value':uu_4,
    'm_cat_id':uu_5,
    'm_cat_value':uu_6,
    's_cat_id':uu_1,
    's_cat_value':uu_7,
    '出现次数':uu_2
    
}
k1_2=pd.DataFrame(data3)
# 降序
k1_3=k1_2.sort_values(by='出现次数',ascending=False)

E:\My large programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [109]:
# 
k1_3.to_excel(r'C:\Users\Administrator\Desktop\POI_Gum_Weak_frequency.xlsx')

In [110]:
kio=k1_3.head(15)
kio.to_excel(r'C:\Users\Administrator\Desktop\Top_15_POI_Gum_Weak_frequency.xlsx')